# 3_박물관 + 미술관 수 및 위치정보 추출

### 순서
1. 데이터 불러오기
    - csv파일을 데이터프레임 형식으로 불러오기
    
    
2. 데이터 정리
    - 영업상태명이 페업인 행을 삭제
    - 소재지전체주소가 없는 행은 2개이므로 도로명전체주소를 통해 찾아서 기입
    - 좌표정보가 있는 행을 위경도로 변환
    - 위경도가 없는 주소를 소재지전체주소를 통해 찾아서 기입


3. 각 지역별 박물관/미술관 수 추출 및 저장
    - 시도명/ 시군구 컬럼을 그룹으로 박물관/미술관 개수 확인
    

4. 각 지역별 박물관/미술관 정보(상호명 및 경/위도) 추출 및 저장
    - 각 상호명 별 경/위도 확인

### 필요한 모듈 import

In [8]:
# 넘파이, 판다스, 플롯관련 패키지 모듈
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
# 모듈 import

from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

import time

In [10]:
service = ChromeService(executable_path=ChromeDriverManager().install())

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:00<00:00, 12.2MB/s]


In [11]:
# 여러 변수 출력 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [12]:
# 경고무시
import warnings
warnings.filterwarnings('ignore')

In [13]:
# 그래프 한글 표시 설정
import platform
from matplotlib import font_manager, rc

plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')

## 1. 데이터 불러오기

### csv파일 불러오기

In [14]:
data=pd.read_csv('data/3/박물관,미술관.csv',encoding='cp949')
data.head()

,번호,개방서비스명,개방서비스아이디,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,...,박물관미술관유형명,폐관일자,폐관사유,자본금,소요자금,법인설립목적,허가조건,법인해산일자,법인명,Unnamed: 42
0,1,"박물관, 미술관",03_07_05_P,4640000,CDFE3232002014000001,2013-07-24,NaN,3,폐업,3,...,미술관,20220228.0,운영상의 어려움,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2,"박물관, 미술관",03_07_05_P,4640000,CDFE3231002014000002,2014-12-16,NaN,3,폐업,3,...,NaN,20190604.0,자진폐관,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,3,"박물관, 미술관",03_07_05_P,3940000,CDFE3231002003100003,2006-01-31,NaN,3,폐업,3,...,전문박물관,NaN,자료 중복으로 폐업처리(2021.6.9.),NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,"박물관, 미술관",03_07_05_P,3940000,CDFE3231002009100006,2009-12-01,NaN,3,폐업,3,...,전시관,20110927.0,보금자리 주택부지로 수용,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,"박물관, 미술관",03_07_05_P,3940000,CDFE3231002020100006,2020-06-25,NaN,3,폐업,3,...,전시관,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- 데이터 확인하기

In [15]:
# 데이터정보
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1211 entries, 0 to 1210
Data columns (total 43 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   번호           1211 non-null   int64  
 1   개방서비스명       1211 non-null   object 
 2   개방서비스아이디     1211 non-null   object 
 3   개방자치단체코드     1211 non-null   int64  
 4   관리번호         1211 non-null   object 
 5   인허가일자        1211 non-null   object 
 6   인허가취소일자      0 non-null      float64
 7   영업상태구분코드     1211 non-null   int64  
 8   영업상태명        1211 non-null   object 
 9   상세영업상태코드     1211 non-null   int64  
 10  상세영업상태명      1211 non-null   object 
 11  폐업일자         62 non-null     object 
 12  휴업시작일자       0 non-null      float64
 13  휴업종료일자       0 non-null      float64
 14  재개업일자        0 non-null      float64
 15  소재지전화        0 non-null      float64
 16  소재지면적        48 non-null     float64
 17  소재지우편번호      0 non-null      float64
 18  소재지전체주소      1209 non-null   object 
 19  도로명전체주

---

## 2. 데이터 정리

### 영업상태명이 페업인 행을 삭제

In [16]:
# 영업상태명이 폐업인 행 삭제
data=data.drop(index=data[data['영업상태명']=='폐업'].index,axis=0)

- 인덱스 설정

In [17]:
# 인덱스 새로 설정
data=data.reset_index(drop=True)

In [18]:
# 뒷 부분 확인
data.tail()

,번호,개방서비스명,개방서비스아이디,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,...,박물관미술관유형명,폐관일자,폐관사유,자본금,소요자금,법인설립목적,허가조건,법인해산일자,법인명,Unnamed: 42
1143,1162,"박물관, 미술관",03_07_05_P,5710000,CDFE3232002012000001,2012-04-03,NaN,1,영업/정상,13,...,미술관,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1144,1163,"박물관, 미술관",03_07_05_P,5710000,CDFE3232002012000002,2001-09-25,NaN,1,영업/정상,13,...,미술관,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1145,1164,"박물관, 미술관",03_07_05_P,5710000,CDFE3232002006000001,2005-12-28,NaN,1,영업/정상,13,...,전시관,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1146,1165,"박물관, 미술관",03_07_05_P,3910000,CDFE3232002022000001,2022-11-01,NaN,1,영업/정상,13,...,미술관,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1147,1166,"박물관, 미술관",03_07_05_P,4050000,CDFE3231002022000001,2022-07-25,NaN,1,영업/정상,13,...,전문박물관,NaN,NaN,0.0,NaN,NaN,NaN,NaN,단국대학교,NaN


### 소재지전체주소가 없는 행은 2개이므로 도로명전체주소를 통해 찾아서 기입

In [19]:
data[data['소재지전체주소'].isna()==True].iloc[:,[18,19]]

,소재지전체주소,도로명전체주소
427,NaN,세종특별자치시 연동면 청연로 492-14
429,NaN,세종특별자치시 금남면 산림박물관길 110


In [20]:
# 직접 기입
data.iloc[427,18]='세종특별자치시 연동면 내판리 152'
data.iloc[429,18]='세종특별자치시 금남면 도남리 12-2'

In [21]:
# 확인
data.iloc[427:430,[18,19]]

,소재지전체주소,도로명전체주소
427,세종특별자치시 연동면 내판리 152,세종특별자치시 연동면 청연로 492-14
428,세종특별자치시 전동면 청송리 6번지,세종특별자치시 전동면 배일길 90-43
429,세종특별자치시 금남면 도남리 12-2,세종특별자치시 금남면 산림박물관길 110


###  좌표정보가 있는 행을 위경도로 변환한다.

In [22]:
data.iloc[:,[26,27]]

,좌표정보(x),좌표정보(y)
0,173068.250849,476530.206203
1,214556.547477,429134.312269
2,NaN,NaN
3,186462.914236,421868.125980
4,253858.607781,433088.552407
...,...,...
1143,243666.139010,349370.133445
1144,NaN,NaN
1145,238642.289356,347376.098957
1146,187044.581473,387931.600858


- 좌표계를 (위도, 경도) 좌표계로 변환

    - pyproj 모듈을 사용하여 좌표계 변환
    - pyproj 모듈 설치
        - pip install pyproj

In [23]:
!pip install pyproj

Defaulting to user installation because normal site-packages is not writeable


In [24]:
from pyproj import Proj, transform

In [25]:
def convert(x,y):
    inProj=Proj(init='epsg:5181')   #투영좌표계
    outProj=Proj(init='epsg:4326')  #지리좌표계
    lon, lat=transform(inProj,outProj,x,y) #x2=경도,y2=위도
    return [lat,lon] 

In [26]:
data_loc=data.iloc[:,[26,27]].apply(lambda pos: convert(pos['좌표정보(x)'],pos['좌표정보(y)']),axis=1)

In [27]:
data_loc[15:40]

15                                  [inf, inf]
16         [37.64185258058, 127.2299703946179]
17     [37.20438936999361, 127.35609012642982]
18     [37.64311881153575, 126.86502972041086]
19     [37.54270374157228, 127.20533484620532]
20     [37.59295385512814, 127.33656602754198]
21     [37.66745596193007, 126.77925957238051]
22     [37.26531997289122, 127.10894382990998]
23       [37.4323210700434, 127.0048643687266]
24                                  [inf, inf]
25     [37.29753175643428, 127.21714711917863]
26     [37.33368613724654, 127.17693653809312]
27     [37.29753175643428, 127.21714711917863]
28     [37.25660483493357, 127.12235831921313]
29     [37.318587538103664, 127.1264186525918]
30                                  [inf, inf]
31    [37.587246515506514, 127.16771133914057]
32     [37.31157644716068, 126.95276580019052]
33     [37.83372959321481, 126.96725829513058]
34     [37.39557976139431, 127.60823595229367]
35     [37.59398981168287, 127.35400487203886]
36     [37.69

In [28]:
# ,으로 나누기
str(data_loc).split(',')

['0        [37.78815319402411',
 ' 126.69425144065035]\n1         [37.36140130296444',
 ' 127.1643161569406]\n2                                     [inf',
 ' inf]\n3         [37.29594627700233',
 ' 126.8473241509708]\n4        [37.39557976139431',
 ' 127.60823595229367]\n                          ...                   \n1143    [36.641774060023565',
 ' 127.48828365603208]\n1144                                  [inf',
 ' inf]\n1145    [36.624022577038936',
 ' 127.43200717110504]\n1146     [36.99016689140651',
 ' 126.85447136862818]\n1147     [37.318587538103664',
 ' 127.1264186525918]\nLength: 1148',
 ' dtype: object']

In [29]:
# 첫 값을 '위도'컬럼으로, 두번째 값을 '경도'컬럼으로 지정한다.
data['위도']=data_loc.str.get(0)
data['경도']=data_loc.str.get(1)

In [30]:
# 데이터 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1148 entries, 0 to 1147
Data columns (total 45 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   번호           1148 non-null   int64  
 1   개방서비스명       1148 non-null   object 
 2   개방서비스아이디     1148 non-null   object 
 3   개방자치단체코드     1148 non-null   int64  
 4   관리번호         1148 non-null   object 
 5   인허가일자        1148 non-null   object 
 6   인허가취소일자      0 non-null      float64
 7   영업상태구분코드     1148 non-null   int64  
 8   영업상태명        1148 non-null   object 
 9   상세영업상태코드     1148 non-null   int64  
 10  상세영업상태명      1148 non-null   object 
 11  폐업일자         0 non-null      object 
 12  휴업시작일자       0 non-null      float64
 13  휴업종료일자       0 non-null      float64
 14  재개업일자        0 non-null      float64
 15  소재지전화        0 non-null      float64
 16  소재지면적        45 non-null     float64
 17  소재지우편번호      0 non-null      float64
 18  소재지전체주소      1148 non-null   object 
 19  도로명전체주

In [31]:
#위도, 경도 컬럼 생성 완료
data.head(3)

,번호,개방서비스명,개방서비스아이디,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,...,폐관사유,자본금,소요자금,법인설립목적,허가조건,법인해산일자,법인명,Unnamed: 42,위도,경도
0,19,"박물관, 미술관",03_07_05_P,6410000,B04012012100005,2012-04-23,NaN,1,영업/정상,1,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,37.788153,126.694251
1,20,"박물관, 미술관",03_07_05_P,6410000,B04012008100004,2008-04-08,NaN,1,영업/정상,1,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,37.361401,127.164316
2,21,"박물관, 미술관",03_07_05_P,6410000,B04012008100006,2007-07-09,NaN,1,영업/정상,1,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,inf,inf


### 위경도가 없는 주소를 소재지전체주소를 통해 찾아서 기입

- 주소를 입력 받아 위경도 정보를 반환하는 함수 생성

In [32]:
# 함수 생성
def find_coor(addr):
    # 크롬 드라이버 객체 생성
    driver = webdriver.Chrome(service=service)
    driver.implicitly_wait(5)
    
    # 웹페이지 로드
    url = 'https://address.dawul.co.kr/'
    driver.get(url)

    # 팝업창 닫기
    tabs = driver.window_handles      # 웹페이지를 로드했을 때 뜨는 모든 창에 대한 정보를 받아옴
                                      # tabs[0]은 주소전환 사이트, tabs[1]은 팝업창
    driver.switch_to.window(tabs[1])  # 팝업창으로 이동
    driver.close()                    # 팝업창 닫기
    driver.switch_to.window(tabs[0])  # 다시 사이트로 이동
    
    # 검색창에 주소 입력    
    search = driver.find_element(By.ID, "input_juso")
    search.send_keys(addr)
    
    # 엔터키 입력
    search.send_keys(Keys.ENTER)
    
    # x, y 좌표 추출
    time.sleep(1)
    coor = driver.find_element(By.ID, 'insert_data_5').text
    
    # 웹페이지 닫기
    driver.close()
    
    # x, y 좌표를 따로 추출하여 반환
    try:
        temp = coor.split(',')
        long = float(temp[0].split(':')[1])
        lat = float(temp[1].split(':')[1])
        coor = [lat, long]
    except:  # 좌표정보를 찾지 못하는 경우
        return '검색 실패'
    
    return coor

In [33]:
coor_list = []

for i in range(data.shape[0]):
    if np.isinf(data['위도'][i]):  # 좌표정보가 결측치인 경우
        coor = find_coor(data.loc[i, '소재지전체주소'])  # 웹 크롤링을 통해 좌표정보를 받아와 입력
        coor_list.append(coor)
        
        print(i, coor)  # 코드가 잘 돌아가는지 실시간으로 확인하기 위해 print
    else:
        coor_list.append("")

2 [37.4859817, 126.795147]
8 [37.7752508, 127.139305]
11 [37.5579093, 127.330341]
14 [37.7305852, 126.711717]
15 [37.4755419, 126.781689]
24 [37.7092688, 126.904194]
30 [37.2211397, 127.199864]
45 [37.2130763, 126.974074]
60 [37.5185965, 127.174802]
64 [37.4183677, 126.918296]
66 [35.9225107, 128.544696]
93 [35.8531773, 128.575917]
102 [37.1761314, 128.209491]
126 [33.4850915, 126.746918]
164 [33.2549074, 126.40983]
206 [38.2123479, 128.515104]
234 [35.1247744, 129.093272]
269 [36.3964516, 127.140586]
283 [36.9884357, 127.907751]
324 [37.4394498, 126.654729]
338 [34.8065359, 127.841738]
351 [35.1843405, 128.555112]
395 [35.2067699, 128.569276]
400 [34.8702116, 128.040752]
434 [33.4931705, 126.955792]
442 [35.1444619, 126.907241]
479 [34.4848498, 126.259305]
499 [34.8444869, 126.249859]
537 [35.909147, 128.807586]
553 [35.85213, 129.268547]
570 [36.5283502, 127.057089]
573 [37.062773, 127.175277]
576 [33.3405867, 126.270845]
577 [34.3909843, 126.250565]
578 [37.5475027, 129.106263]
580 

In [34]:
for i in range(data.shape[0]):
    if type(coor_list[i]) == list:
        data.loc[i, '위도'] = coor_list[i][0]  # coor_list에 있던 위경도 정보를 data에 대입
        data.loc[i, '경도'] = coor_list[i][1]

In [35]:
data.위도
data.경도

0       37.788153
1       37.361401
2       37.485982
3       37.295946
4       37.395580
          ...    
1143    36.641774
1144    36.656187
1145    36.624023
1146    36.990167
1147    37.318588
Name: 위도, Length: 1148, dtype: float64

0       126.694251
1       127.164316
2       126.795147
3       126.847324
4       127.608236
           ...    
1143    127.488284
1144    127.492370
1145    127.432007
1146    126.854471
1147    127.126419
Name: 경도, Length: 1148, dtype: float64

----------

## 3. 각 지역별 박물관 및 미술관 수 추출 및 저장

- split를 이용해 시도명/ 시군구컬럼 생성

In [36]:
# 소재지전체주소의 첫 값을 시도명으로 생성
data['시도명']=data['소재지전체주소'].str.split(" ",expand=True)[0]
data.head(5)

,번호,개방서비스명,개방서비스아이디,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,...,자본금,소요자금,법인설립목적,허가조건,법인해산일자,법인명,Unnamed: 42,위도,경도,시도명
0,19,"박물관, 미술관",03_07_05_P,6410000,B04012012100005,2012-04-23,NaN,1,영업/정상,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,37.788153,126.694251,경기도
1,20,"박물관, 미술관",03_07_05_P,6410000,B04012008100004,2008-04-08,NaN,1,영업/정상,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,37.361401,127.164316,경기도
2,21,"박물관, 미술관",03_07_05_P,6410000,B04012008100006,2007-07-09,NaN,1,영업/정상,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,37.485982,126.795147,경기도
3,22,"박물관, 미술관",03_07_05_P,6410000,B04012008100007,2008-08-06,NaN,1,영업/정상,1,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,37.295946,126.847324,경기도
4,23,"박물관, 미술관",03_07_05_P,6410000,B04012008100008,2000-10-17,NaN,1,영업/정상,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,37.395580,127.608236,경기도


In [37]:
# 소재지전체주소의 두번째 값을 시도명으로 생성
data['시군구명']=data['소재지전체주소'].str.split(" ",expand=True)[1]
data.head(5)

,번호,개방서비스명,개방서비스아이디,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,...,소요자금,법인설립목적,허가조건,법인해산일자,법인명,Unnamed: 42,위도,경도,시도명,시군구명
0,19,"박물관, 미술관",03_07_05_P,6410000,B04012012100005,2012-04-23,NaN,1,영업/정상,1,...,NaN,NaN,NaN,NaN,NaN,NaN,37.788153,126.694251,경기도,파주시
1,20,"박물관, 미술관",03_07_05_P,6410000,B04012008100004,2008-04-08,NaN,1,영업/정상,1,...,NaN,NaN,NaN,NaN,NaN,NaN,37.361401,127.164316,경기도,광주시
2,21,"박물관, 미술관",03_07_05_P,6410000,B04012008100006,2007-07-09,NaN,1,영업/정상,1,...,NaN,NaN,NaN,NaN,NaN,NaN,37.485982,126.795147,경기도,부천시
3,22,"박물관, 미술관",03_07_05_P,6410000,B04012008100007,2008-08-06,NaN,1,영업/정상,1,...,0.0,NaN,NaN,NaN,NaN,NaN,37.295946,126.847324,경기도,안산시
4,23,"박물관, 미술관",03_07_05_P,6410000,B04012008100008,2000-10-17,NaN,1,영업/정상,1,...,NaN,NaN,NaN,NaN,NaN,NaN,37.395580,127.608236,경기도,여주군


- 지역별 개수 추출

In [38]:
# 지역별 개수 추출 데이터프레임 추출
df_count = data.groupby(['시도명','시군구명']).size().reset_index(name='박물관/미술관 수')
df_count

,시도명,시군구명,박물관/미술관 수
0,강원도,강릉시,24
1,강원도,고성군,2
2,강원도,동해시,1
3,강원도,삼척시,3
4,강원도,속초시,4
...,...,...,...
190,충청북도,증평군,1
191,충청북도,진천군,2
192,충청북도,청원군,1
193,충청북도,청주시,29


In [39]:
df_count.isnull().sum()

시도명          0
시군구명         0
박물관/미술관 수    0
dtype: int64

- csv파일 저장

In [40]:
# csv파일로 저장
df_count.to_csv('./data/3/전처리_완료/지역별 박물관+미술관 수.csv', sep=',', na_rep='NaN')

----------

## 4. 각 지역별 박물관/미술관 정보(상호명 및 경/위도) 추출 및 저장

- 필요한 컬럼('사업장명','시도명','시군구명','경도', '위도') 추출

In [41]:
df_loc = data[['사업장명','시도명','시군구명','경도', '위도']]

- csv파일로 저장

In [42]:
# csv파일로 저장
df_loc.to_csv('./data/3/전처리_완료/지역별 박물관+미술관 위치정보.csv', sep=',', na_rep='NaN')